In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import log_loss

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [2]:
#Import and split data
#data = pd.read_csv('../input/voice.csv')
data_train = pd.read_csv('train_features.csv')
data_test = pd.read_csv('test_features.csv')
data_train_target_ns = pd.read_csv('train_targets_nonscored.csv')
data_train_target_s = pd.read_csv('train_targets_scored.csv')
ss = pd.read_csv('sample_submission.csv')


#Preprocess
#Categorize data
def preprocess(df):
    df = df.copy()
    # transform treatment group
    # transform vehicule group
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    
    # transform D1 and D2 group
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    
    # # transform 
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
    del df['sig_id']
    return df



train = preprocess(data_train)
test = preprocess(data_test)
del data_train_target_s['sig_id']
top_features = list(range(1, len(train.columns)))

In [3]:
train.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,0,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,0,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,0,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,0,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
def create_model(num_columns):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(num_columns))
        
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(510, activation='relu')))
        
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(250, activation='relu')))

    #============ Final Layer =================
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid")))

    model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 700), 
                  loss='binary_crossentropy', metrics=['accuracy']
                  )
    return model

In [5]:
def metric(y_true, y_pred):
    metrics = []
    for _target in data_train_target_s.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [12]:
N_STARTS = 9

res = data_train_target_s.copy()
ss.loc[:, data_train_target_s.columns] = 0
res.loc[:, data_train_target_s.columns] = 0

top_feats = list(range(0, 785))
historys = dict()

#print (ss.loc[:, data_train_target_s.columns])

tf.random.set_seed(43)
for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed*3, shuffle=True).split(data_train_target_s, data_train_target_s)):
        print(f"======{data_train_target_s.values[tr].shape}========{data_train_target_s.values[te].shape}=====")
        print(f'Seed: {seed} => Fold: {n}')
        
        model = create_model(len(top_feats))
        
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_lr=1e-5, patience=3, verbose=1, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 1, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        early = EarlyStopping(monitor="val_loss", mode="min", restore_best_weights=True, patience= 5, verbose = 1)
        
        history = model.fit(train.values[tr][:, top_feats], data_train_target_s.values[tr], validation_data=(train.values[te][:, top_feats], data_train_target_s.values[te]),
                  epochs=100, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt, early], verbose=2)
            
        historys[f'history_{seed+1}'] = history
        print("Model History Saved.")
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        ss.loc[:, data_train_target_s.columns] += test_predict
        res.loc[te, data_train_target_s.columns] += val_predict
        
        print(f'OOF Metric For SEED {seed} => FOLD {n} : {metric(data_train_target_s.loc[te, data_train_target_s.columns], pd.DataFrame(val_predict, columns=data_train_target_s.columns))}')
        print('+-' * 10)
    
ss.loc[:, data_train_target_s.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train_target_s.columns] /= N_STARTS

======(20412, 206)========(3402, 206)=====
Seed: 0 => Fold: 0
Epoch 1/100

Epoch 00001: val_loss improved from inf to 0.13697, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.5117 - accuracy: 0.0164 - val_loss: 0.1370 - val_accuracy: 0.0068
Epoch 2/100

Epoch 00002: val_loss improved from 0.13697 to 0.03587, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0718 - accuracy: 0.0232 - val_loss: 0.0359 - val_accuracy: 0.0259
Epoch 3/100

Epoch 00003: val_loss improved from 0.03587 to 0.02496, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0313 - accuracy: 0.0378 - val_loss: 0.0250 - val_accuracy: 0.0617
Epoch 4/100

Epoch 00004: val_loss improved from 0.02496 to 0.02196, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0243 - accuracy: 0.0442 - val_loss: 0.0220 - val_accuracy: 0.0453
Epoch 5/100

Epoch 00005: val_loss improved from 0.02196 to 0.02065, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0222 - accuracy: 0.0453 - val_los

Epoch 43/100

Epoch 00043: val_loss did not improve from 0.01531
160/160 - 3s - loss: 0.0148 - accuracy: 0.1406 - val_loss: 0.0153 - val_accuracy: 0.1223
Epoch 44/100

Epoch 00044: val_loss did not improve from 0.01531
Restoring model weights from the end of the best epoch.
160/160 - 3s - loss: 0.0148 - accuracy: 0.1386 - val_loss: 0.0153 - val_accuracy: 0.1235
Epoch 00044: early stopping
Model History Saved.
OOF Metric For SEED 0 => FOLD 0 : 0.015305935949132215
+-+-+-+-+-+-+-+-+-+-
======(20412, 206)========(3402, 206)=====
Seed: 0 => Fold: 1
Epoch 1/100

Epoch 00001: val_loss improved from inf to 0.14407, saving model to repeat:0_Fold:1.hdf5
160/160 - 3s - loss: 0.5142 - accuracy: 0.0136 - val_loss: 0.1441 - val_accuracy: 0.0024
Epoch 2/100

Epoch 00002: val_loss improved from 0.14407 to 0.03515, saving model to repeat:0_Fold:1.hdf5
160/160 - 2s - loss: 0.0715 - accuracy: 0.0212 - val_loss: 0.0351 - val_accuracy: 0.0511
Epoch 3/100

Epoch 00003: val_loss improved from 0.03515 to 0.0

Epoch 40/100

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1e-05.

Epoch 00040: val_loss improved from 0.01509 to 0.01509, saving model to repeat:0_Fold:1.hdf5
160/160 - 3s - loss: 0.0148 - accuracy: 0.1365 - val_loss: 0.0151 - val_accuracy: 0.1273
Epoch 41/100

Epoch 00041: val_loss did not improve from 0.01509
160/160 - 2s - loss: 0.0149 - accuracy: 0.1371 - val_loss: 0.0151 - val_accuracy: 0.1273
Epoch 42/100

Epoch 00042: val_loss improved from 0.01509 to 0.01509, saving model to repeat:0_Fold:1.hdf5
160/160 - 3s - loss: 0.0148 - accuracy: 0.1369 - val_loss: 0.0151 - val_accuracy: 0.1273
Epoch 43/100

Epoch 00043: val_loss improved from 0.01509 to 0.01509, saving model to repeat:0_Fold:1.hdf5
160/160 - 3s - loss: 0.0148 - accuracy: 0.1376 - val_loss: 0.0151 - val_accuracy: 0.1276
Epoch 44/100

Epoch 00044: val_loss improved from 0.01509 to 0.01508, saving model to repeat:0_Fold:1.hdf5
160/160 - 3s - loss: 0.0148 - accuracy: 0.1359 - val_loss: 0.0151 - val_accuracy: 0.12

Epoch 33/100

Epoch 00033: val_loss improved from 0.01551 to 0.01549, saving model to repeat:0_Fold:2.hdf5
160/160 - 3s - loss: 0.0151 - accuracy: 0.1348 - val_loss: 0.0155 - val_accuracy: 0.1235
Epoch 34/100

Epoch 00034: val_loss improved from 0.01549 to 0.01549, saving model to repeat:0_Fold:2.hdf5
160/160 - 3s - loss: 0.0150 - accuracy: 0.1343 - val_loss: 0.0155 - val_accuracy: 0.1208
Epoch 35/100

Epoch 00035: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00035: val_loss improved from 0.01549 to 0.01544, saving model to repeat:0_Fold:2.hdf5
160/160 - 3s - loss: 0.0149 - accuracy: 0.1350 - val_loss: 0.0154 - val_accuracy: 0.1287
Epoch 36/100

Epoch 00036: val_loss improved from 0.01544 to 0.01539, saving model to repeat:0_Fold:2.hdf5
160/160 - 3s - loss: 0.0147 - accuracy: 0.1408 - val_loss: 0.0154 - val_accuracy: 0.1308
Epoch 37/100

Epoch 00037: val_loss improved from 0.01539 to 0.01535, saving model to repeat:0_Fold:2.hdf5
160/160 - 3s - loss: 0.0146

Epoch 22/100

Epoch 00022: val_loss improved from 0.01620 to 0.01605, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0167 - accuracy: 0.1067 - val_loss: 0.0160 - val_accuracy: 0.1088
Epoch 23/100

Epoch 00023: val_loss improved from 0.01605 to 0.01600, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0165 - accuracy: 0.1117 - val_loss: 0.0160 - val_accuracy: 0.1099
Epoch 24/100

Epoch 00024: val_loss improved from 0.01600 to 0.01592, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0164 - accuracy: 0.1091 - val_loss: 0.0159 - val_accuracy: 0.1099
Epoch 25/100

Epoch 00025: val_loss did not improve from 0.01592
160/160 - 2s - loss: 0.0163 - accuracy: 0.1139 - val_loss: 0.0160 - val_accuracy: 0.1108
Epoch 26/100

Epoch 00026: val_loss improved from 0.01592 to 0.01574, saving model to repeat:0_Fold:3.hdf5
160/160 - 3s - loss: 0.0162 - accuracy: 0.1157 - val_loss: 0.0157 - val_accuracy: 0.1096
Epoch 27/100

Epoch 00027: val_loss improved from 0.01574 to 0

...

Epoch 25/100

Epoch 00025: val_loss improved from 0.01587 to 0.01578, saving model to repeat:8_Fold:3.hdf5
160/160 - 2s - loss: 0.0161 - accuracy: 0.1171 - val_loss: 0.0158 - val_accuracy: 0.1091
Epoch 26/100

Epoch 00026: val_loss improved from 0.01578 to 0.01569, saving model to repeat:8_Fold:3.hdf5
160/160 - 2s - loss: 0.0159 - accuracy: 0.1171 - val_loss: 0.0157 - val_accuracy: 0.1141
Epoch 27/100

Epoch 00027: val_loss improved from 0.01569 to 0.01564, saving model to repeat:8_Fold:3.hdf5
160/160 - 2s - loss: 0.0158 - accuracy: 0.1218 - val_loss: 0.0156 - val_accuracy: 0.1138
Epoch 28/100

Epoch 00028: val_loss improved from 0.01564 to 0.01556, saving model to repeat:8_Fold:3.hdf5
160/160 - 2s - loss: 0.0157 - accuracy: 0.1238 - val_loss: 0.0156 - val_accuracy: 0.1223
Epoch 29/100

Epoch 00029: val_loss improved from 0.01556 to 0.01550, saving model to repeat:8_Fold:3.hdf5
160/160 - 2s - loss: 0.0156 - accuracy: 0.1253 - val_loss: 0.0155 - val_accuracy: 0.1176
Epoch 30/100

Epoch 


Epoch 00008: val_loss improved from 0.01907 to 0.01867, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0195 - accuracy: 0.0673 - val_loss: 0.0187 - val_accuracy: 0.0923
Epoch 9/100

Epoch 00009: val_loss improved from 0.01867 to 0.01834, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0192 - accuracy: 0.0707 - val_loss: 0.0183 - val_accuracy: 0.0726
Epoch 10/100

Epoch 00010: val_loss improved from 0.01834 to 0.01779, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0189 - accuracy: 0.0744 - val_loss: 0.0178 - val_accuracy: 0.0752
Epoch 11/100

Epoch 00011: val_loss improved from 0.01779 to 0.01759, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0188 - accuracy: 0.0787 - val_loss: 0.0176 - val_accuracy: 0.0967
Epoch 12/100

Epoch 00012: val_loss improved from 0.01759 to 0.01744, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0183 - accuracy: 0.0832 - val_loss: 0.0174 - val_accuracy: 0.0864
Epoch 13/100

Epoch 00013: val_los

Epoch 50/100

Epoch 00050: val_loss did not improve from 0.01510
160/160 - 2s - loss: 0.0145 - accuracy: 0.1458 - val_loss: 0.0151 - val_accuracy: 0.1314
Epoch 51/100

Epoch 00051: val_loss improved from 0.01510 to 0.01510, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1446 - val_loss: 0.0151 - val_accuracy: 0.1317
Epoch 52/100

Epoch 00052: val_loss improved from 0.01510 to 0.01510, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1453 - val_loss: 0.0151 - val_accuracy: 0.1317
Epoch 53/100

Epoch 00053: val_loss improved from 0.01510 to 0.01510, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1433 - val_loss: 0.0151 - val_accuracy: 0.1314
Epoch 54/100

Epoch 00054: val_loss improved from 0.01510 to 0.01510, saving model to repeat:8_Fold:4.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1464 - val_loss: 0.0151 - val_accuracy: 0.1326
Epoch 55/100

Epoch 00055: val_loss did not improve from 0.015

Epoch 34/100

Epoch 00034: val_loss did not improve from 0.01534
160/160 - 2s - loss: 0.0150 - accuracy: 0.1376 - val_loss: 0.0153 - val_accuracy: 0.1293
Epoch 35/100

Epoch 00035: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00035: val_loss did not improve from 0.01534
160/160 - 2s - loss: 0.0149 - accuracy: 0.1371 - val_loss: 0.0153 - val_accuracy: 0.1238
Epoch 36/100

Epoch 00036: val_loss improved from 0.01534 to 0.01525, saving model to repeat:8_Fold:5.hdf5
160/160 - 2s - loss: 0.0146 - accuracy: 0.1443 - val_loss: 0.0152 - val_accuracy: 0.1249
Epoch 37/100

Epoch 00037: val_loss improved from 0.01525 to 0.01522, saving model to repeat:8_Fold:5.hdf5
160/160 - 2s - loss: 0.0146 - accuracy: 0.1443 - val_loss: 0.0152 - val_accuracy: 0.1240
Epoch 38/100

Epoch 00038: val_loss improved from 0.01522 to 0.01520, saving model to repeat:8_Fold:5.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1424 - val_loss: 0.0152 - val_accuracy: 0.1243
Epoch 39/100

Epoch 00

Epoch 31/100

Epoch 00031: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00031: val_loss improved from 0.01576 to 0.01574, saving model to repeat:8_Fold:6.hdf5
160/160 - 2s - loss: 0.0154 - accuracy: 0.1263 - val_loss: 0.0157 - val_accuracy: 0.1190
Epoch 32/100

Epoch 00032: val_loss improved from 0.01574 to 0.01564, saving model to repeat:8_Fold:6.hdf5
160/160 - 2s - loss: 0.0152 - accuracy: 0.1292 - val_loss: 0.0156 - val_accuracy: 0.1217
Epoch 33/100

Epoch 00033: val_loss improved from 0.01564 to 0.01560, saving model to repeat:8_Fold:6.hdf5
160/160 - 2s - loss: 0.0151 - accuracy: 0.1335 - val_loss: 0.0156 - val_accuracy: 0.1235
Epoch 34/100

Epoch 00034: val_loss improved from 0.01560 to 0.01557, saving model to repeat:8_Fold:6.hdf5
160/160 - 2s - loss: 0.0151 - accuracy: 0.1324 - val_loss: 0.0156 - val_accuracy: 0.1226
Epoch 35/100

Epoch 00035: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00035: val_loss improved from 0.

In [13]:
print(f'OOF Metric: {metric(data_train_target_s, res)}')

OOF Metric: 0.015101734847319779


In [1]:
res.loc[train['cp_type']==1, data_train_target_s.columns] = 0
print(f'OOF Metric After Processing : {metric(data_train_target_s, res)}')

NameError: name 'res' is not defined

In [ ]:
ss.loc[test['cp_type']==1, data_train_target_s.columns] = 0

In [ ]:
ss.to_csv('submission.csv', index=False)